In [2]:
import pandas as pd
import numpy as np
import folium
import sys
from pathlib import Path

In [3]:
current_dir = Path.cwd()
src_path = current_dir.parent / 'src'
sys.path.append(str(src_path))

from config import INPUT_FILE

In [11]:
df = pd.read_csv(INPUT_FILE).reset_index()
display(df.head())

,index,url,title_x,price_php,bedrooms,bathrooms,floor_area_sqm,location_x,latitude,longitude,...,amenities,scrape_status,property_id,title_y,location_y,price_preview,bedrooms_preview,bathrooms_preview,city,property_type
0,0,https://www.lamudi.com.ph/property/41032-73-bc...,2BR Condo Unit for Rent at High Park Vertis No...,75000.0,2.0,2.0,NaN,"Diliman, Quezon City",14.648828,121.038245,...,"Balcony, Fully furnished, Air conditioning, Se...",success,01976436-2d51-791a-bb27-bccf89fc5404,Condo For Rent in Diliman,"Diliman, Quezon City",75000.0,2.0,2.0,quezon-city,condo
1,1,https://www.lamudi.com.ph/property/41032-73-79...,2 Bedroom Condo near Solaire Resort Quezon City,40000.0,2.0,1.0,NaN,"Project 7, Quezon City",14.649498,121.034991,...,"Air conditioning, Equipped kitchen, Unfurnishe...",success,aad132b0-d05a-1007-ade2-79ffb3c5a5b2,Condo For Sale in Project 7,"Project 7, Quezon City",40000.0,2.0,1.0,quezon-city,condo
2,2,https://www.lamudi.com.ph/property/41032-73-36...,"3-BR Condo for Rent at Royal Mansion, Wack Wac...",60000.0,3.0,3.0,NaN,"Wack-Wack Greenhills, Mandaluyong",14.586397,121.050040,...,"Balcony, Air conditioning, Fully furnished, Li...",success,01976c22-4298-70c0-9c21-364ce8ef565b,Condo For Rent in Wack-Wack Greenhills,"Wack-Wack Greenhills, Mandaluyong",60000.0,3.0,3.0,mandaluyong,condo
3,3,https://www.lamudi.com.ph/property/41032-73-e3...,"For RENT: 1-BR, The Residences at Greenbelt, ...",76000.0,1.0,1.0,57.0,"Antonio Arnaiz Avenue, Makati",14.551113,121.020071,...,"Equipped kitchen, Water, Fully furnished, Air ...",success,019a4245-6e16-71fd-b90b-e316051eff02,Condo For Rent in Antonio Arnaiz Avenue,"Antonio Arnaiz Avenue, Makati",76000.0,1.0,1.0,makati,condo
4,4,https://www.lamudi.com.ph/property/41032-73-81...,Unfurnished 2BR Condo Trees Residences near SM...,15600.0,2.0,1.0,30.0,"Kaligayahan, Quezon City",14.732115,121.049436,...,"Air conditioning, Service room, Water, Unfurni...",success,01981775-957a-730c-a759-818f2a4a99aa,Condo For Rent in Kaligayahan,"Kaligayahan, Quezon City",15600.0,2.0,1.0,quezon-city,condo


In [12]:
# HTML requests
import requests
import xml.etree.ElementTree as ET

def get_train_data(train_id):
    """
    Fetches data for a given train ID from OpenStreetMap and returns a Pandas DataFrame.

    Args:
        train_id (str): The OpenStreetMap relation ID for the train route.

    Returns:
        pandas.DataFrame: A DataFrame containing the train route data, or None if an error occurs.
    """
    # Fetch the XML data
    url = f'https://www.openstreetmap.org/api/0.6/relation/{train_id}'

    try:

        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        # Parse the XML response
        root = ET.fromstring(response.content)

        # Initialize lists to store data
        stop_ids = []
        alt_names = []
        names = []
        starts = []
        ends = []
        lats = []
        lons = []
        stations_names = []
        stations_ids = []

        # Find the relation element
        for relation in root.findall('relation'):
            # Iterate over all member elements
            for member in relation.findall('member'):
                type_ = member.get('type')
                ref = member.get('ref')
                if type_ == 'node':
                    stop_ids.append(ref)
                    lat, lon, station_name, station_id = get_node_coordinates(ref)
                    lats.append(lat)
                    lons.append(lon)
                    stations_names.append(station_name)
                    stations_ids.append(station_id)

            # Extract tag values from the relation
            alt_name = None
            name = None
            start = None
            end = None
            for tag in relation.findall('tag'):
                k = tag.get('k')
                v = tag.get('v')
                if k == 'alt_name':
                    alt_name = v
                elif k == 'name':
                    name = v
                elif k == 'from':
                    start = v
                elif k == 'to':
                    end = v

            # Extend tag values to match the number of stops
            num_stops = len(stop_ids) - len(alt_names)
            alt_names.extend([alt_name] * num_stops)
            names.extend([name] * num_stops)
            starts.extend([start] * num_stops)
            ends.extend([end] * num_stops)

        # Create a DataFrame
        data = {
            'Stop ID': stop_ids,
            'Alt Name': alt_names,
            'Lat': lats,
            'Lon': lons,
            'Station Name': stations_names,
            'Station ID': stations_ids,
            'Name': names,
            'Start': starts,
            'End': ends
        }
        df = pd.DataFrame(data)

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for train ID {train_id}: {e}")
        return None
    except ET.ParseError as e:
        print(f"Error parsing XML for train ID {train_id}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for train ID {train_id}: {e}")
        return None

def get_node_coordinates(node_id):
    """
    Fetches the coordinates of a node from OpenStreetMap.

    Args:
        node_id (str): The OpenStreetMap node ID.

    Returns:
        tuple: A tuple containing (latitude, longitude, station name, station ID), or (None, None, None, None) if an error occurs.
    """
    url = f"https://api.openstreetmap.org/api/0.6/node/{node_id}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        root = ET.fromstring(response.content)

        node = root.find('node')
        if node is not None:
            lat = node.get('lat')
            lon = node.get('lon')
            station_name = None
            station_id = None
            for tag in node.findall('tag'):
                k = tag.get('k')
                v = tag.get('v')
                if k == 'name':
                    station_name = v
                elif k == 'ref':
                    station_id = v
            return (
                float(lat) if lat else None,
                float(lon) if lon else None,
                station_name,
                station_id
            )
        return (None, None, None, None)

    except (requests.RequestException, ET.ParseError, ValueError) as e:
        print(f"Error processing node {node_id}: {str(e)}")
        return (None, None, None, None)

In [13]:
from shapely.geometry import Point, Polygon, LineString
# Train lines (OSM relation IDs)
TRAIN_LINES = {
    'MRT Line 3': 109159
}

# Fetch station data for each line
all_stations = []
for line_name, train_id in TRAIN_LINES.items():
    print(f"📡 Fetching {line_name} (ID: {train_id})...", end=' ')

    train_df = get_train_data(str(train_id))

    if train_df is not None:
        train_df['train_id'] = train_id
        train_df['line_name'] = line_name
        all_stations.append(train_df)
        print(f"✅ {len(df)} stations")
    else:
        print("❌ Failed")

# Combine all lines
train_stations = pd.concat(all_stations, ignore_index=True)

# Create Point geometries from coordinates
train_stations['geometry'] = train_stations.apply(
    lambda row: Point(row['Lon'], row['Lat'])
    if pd.notna(row['Lat']) and pd.notna(row['Lon'])
    else None,
    axis=1
)

# Remove invalid coordinates
train_stations = train_stations.dropna(subset=['geometry']).reset_index(drop=True)

print(f"\n✅ Total: {len(train_stations)} stations across {len(TRAIN_LINES)} lines")
train_stations

📡 Fetching MRT Line 3 (ID: 109159)... ✅ 1300 stations

✅ Total: 14 stations across 1 lines


,Stop ID,Alt Name,Lat,Lon,Station Name,Station ID,Name,Start,End,train_id,line_name,geometry
0,9223445517,None,14.653975,121.031079,New North Avenue,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0310785 14.6539745)
1,6264846311,None,14.651670,121.032594,North Avenue,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0325938 14.6516697)
2,6264846322,None,14.642437,121.038625,Quezon Avenue,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0386254 14.6424366)
3,88775861,None,14.635333,121.043275,GMA Kamuning,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0432749 14.635333)
4,11328560046,None,14.619478,121.051057,Cubao,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.051057 14.6194777)
5,6264796043,None,14.607541,121.056574,Santolan-Annapolis,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0565742 14.6075407)
6,5397307747,None,14.587341,121.056519,Ortigas,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0565192 14.5873409)
7,11328574094,None,14.581102,121.053408,Shaw Boulevard,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0534077 14.5811015)
8,320131141,None,14.573093,121.047644,Boni,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0476442 14.5730934)
9,5397307750,None,14.566719,121.045438,Guadalupe,None,MRT Line 3: North Avenue → Taft Avenue,North Avenue,Taft Avenue,109159,MRT Line 3,POINT (121.0454381 14.5667194)


In [14]:
df.columns

Index(['index', 'url', 'title_x', 'price_php', 'bedrooms', 'bathrooms',
       'floor_area_sqm', 'location_x', 'latitude', 'longitude', 'description',
       'furnishing', 'amenities', 'scrape_status', 'property_id', 'title_y',
       'location_y', 'price_preview', 'bedrooms_preview', 'bathrooms_preview',
       'city', 'property_type'],
      dtype='object')

In [ ]:
# MAP INITIALIZATION
# Clean data first - remove any NaN values
dff = df.dropna(subset=['latitude', 'longitude', 'price_php', 'floor_area_sqm'])

data_clean = dff[(dff['price_php'] < 35000) & (dff['price_php'] > 10000)]

# Create base map centered on the mean coordinates
center_lat = data_clean['latitude'].mean()
center_lon = data_clean['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# COLOR SCALE SETUP
# Create color gradient based on price with proper quantiles
price_min = data_clean['price_php'].min()
price_max = data_clean['price_php'].max()

# Check if we have valid price range
if price_min == price_max:
    # If all prices are the same, use a small range
    price_scale = folium.LinearColormap(
        colors=['green', 'yellow', 'red'],
        vmin=price_min * 0.9,
        vmax=price_max * 1.1,
        caption='Rent Price (PHP)'
    )
else:
    price_scale = folium.LinearColormap(
        colors=['green', 'yellow', 'red'],
        vmin=price_min,
        vmax=price_max,
        caption='Rent Price (PHP)'
    )

# ADD PROPERTY MARKERS
for index, row in data_clean.iterrows():
    # Skip if price is invalid
    if pd.isna(row['price_php']) or not np.isfinite(row['price_php']):
        continue

    # Create popup content with property details
    popup_html = f"""
    <div style='font-size: 14px; font-family: Arial, sans-serif; min-width: 250px;'>
        <strong>Lot ID:</strong> {row.get('index', 'N/A')}<br>
        <p style='font-weight: bold; margin: 10px 0;'>{row['title_x']}</p>
        <hr style='margin: 10px 0; border: none; border-top: 1px solid #ccc;'>
        <strong>Price:</strong> {row['price_php']:,.2f} PHP<br><br>
        <a href="{row['url']}" target="_blank" rel="noopener noreferrer"
           style='color: #0066cc; text-decoration: none;'>
            📋 View Listing →
        </a>
    </div>
    """

    # Create tooltip with brief info
    tooltip_text = f"<strong>{row['title_x']}</strong><br>{row['price_php']:,.2f} PHP"

    # Add circle marker to map
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=price_scale(row['price_php']),
        fill=True,
        fill_color=price_scale(row['price_php']),
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=tooltip_text
    ).add_to(m)

# ADD TRAIN STATION MARKERS
if 'train_stations' in locals() and not train_stations.empty:
    for idx, station in train_stations.iterrows():
        if station['geometry'] and station['Lat'] and station['Lon']:
            folium.Marker(
                location=[station['Lat'], station['Lon']],
                icon=folium.Icon(color='red', icon='train', prefix='fa'),
                tooltip=f"Train Station: {station['Station Name']}",
                popup=f"<strong>{station['Station Name']}</strong><br>{station['line_name']}"

            ).add_to(m)

folium.Marker(
    location=[14.584678675972874, 121.05920053403466],
    popup='Podium West Building (Destination)',
    icon=folium.Icon(color='purple', icon='info-sign')
).add_to(m)

# FINALIZE MAP
# Add color scale legend to map
m.add_child(price_scale)

# Save and display the map
#m.save('property_map.html')
m